In [53]:
from skypy.utils import package_directory, load_yaml, merge_two_dictionaries, add_to_pickel_file
from skypy.telescopes.hst import HubbleObservation, get_hst_ceres_fluxes_hourly
from skypy.telescopes.ceres import prep_data
from skypy.tles import pick_the_right_tle

import pandas as pd
import os.path as path
import numpy as np
import pickle

### WARNING ### you must download the ceres file FIRST using the example script "download_ceres_data"

In [54]:
# Define where the MAST data is kept
data_path = package_directory('data', 'MDRIZSKY.csv')

In [55]:
# Define pickle path to find the stored ceres data
pickel_path = "/Users/physarah/Desktop"

In [56]:
# Define the save pickle path for each of the dictionaries created in this loop
save_dictionaries_path = "/Users/physarah/Desktop"

In [57]:
# Define where the master TLE yaml file is
tle_dictionary = load_yaml(package_directory('data', 'tle_dictionary.yml'))

In [58]:
# Parse MAST data into pandas dataframe
master_data_frame = pd.read_csv(data_path)

In [59]:
# Choose only exposures that are taken during the day
data_sun_selection = master_data_frame.loc[master_data_frame['SUN_ALT'] > 0]

In [60]:
# Choose only GOODS fields for now
data_goods_selection = data_sun_selection[data_sun_selection['Target Name'].str.contains(
    "GOODS")]

In [61]:
# Choose only exposures in the X bandpass (here is it F606W)
data_final_filter_selection = data_goods_selection[data_goods_selection['FILTER1'].str.contains(
    "F850LP")]

In [62]:
# Reset the index of the data set
data_final_selection = data_final_filter_selection
df_to_use = data_final_selection.reset_index()

In [63]:
# Pick the exposure to use in the list of Hubble obs
i = 33

In [64]:
# Init the HSTObs class for this observation
hst_obs = HubbleObservation(df_to_use.loc[i])

In [65]:
# Get the expected file name for this exposure
pickel_filename = "FF850LP_{}".format(hst_obs.mid_times.strftime("%d%m%y_%H%M%S"))
total_pickle_filepath = pickel_path + "/" + pickel_filename + ".pickle"

In [66]:
# Check if this exists
if not path.exists(total_pickle_filepath):
    print("Skipping this one because the file doesn't exist: {}".format(total_pickle_filepath))
    pass

else:
    # Open the ceres dictionary
    try:
        pickle_data = pickle.load(open(total_pickle_filepath, 'rb'))
        ceres_fluxes_dict = pickle_data['sw_upward_flux'][0, :, :]

        # Get the corresponding TLE
        correct_tle = pick_the_right_tle(hst_obs.start_times, tle_dictionary)

        if not correct_tle:
            pass

        else:
            # Put the data into a dataframe and add back the lon and lat
            data_frame_organised = prep_data(ceres_fluxes_dict)

            # Get the fluxes under hst
            fluxes_in_start, fluxes_in_end, fluxes_in_mid = get_hst_ceres_fluxes_hourly(hst_obs,
                                                                                        correct_tle,
                                                                                        data_frame_organised)

            # Compute the zodi strength
            try:
                hst_zodi = hst_obs.zodi_strength.value
            except:
                hst_zodi = -999

            # Compute the average flux under hst
            flux_start_av = np.average(fluxes_in_start)
            flux_mid_av = np.average(fluxes_in_mid)
            flux_end_av = np.average(fluxes_in_end)

            # Compute the standard deviation in flux
            flux_start_stdv = np.std(fluxes_in_start)
            flux_mid_stdv = np.std(fluxes_in_mid)
            flux_end_stdv = np.std(fluxes_in_end)

            # Put it all in a dictionary
            pickel_dict = {'f_s_av': flux_start_av,
                           'f_m_av': flux_mid_av,
                           'f_e_av': flux_end_av,
                           'f_s_stdv': flux_start_stdv,
                           'f_m_stdv': flux_mid_stdv,
                           'f_e_stdv': flux_end_stdv,
                           'zodi': hst_zodi}

            # Put everything into the one dictionary
            final_pickel = merge_two_dictionaries(
                df_to_use.loc[i].to_dict(), pickel_dict)

            # Save to pickle file
            add_to_pickel_file(pickel_path=save_dictionaries_path,
                               pickel_filename=(pickel_filename + "arc_dict.pickle"),
                               pickel_dictionary=final_pickel)
            print("File saved :)")
    except:
        pickel_dict = {'f_s_av': -999,
                       'f_m_av': -999,
                       'f_e_av': -999,
                       'f_s_stdv': -999,
                       'f_m_stdv': -999,
                       'f_e_stdv': -999,
                       'zodi': -999}
        # Put everything into the one dictionary
        final_pickel = merge_two_dictionaries(
            df_to_use.loc[i].to_dict(), pickel_dict)

        # Save to pickle file
        add_to_pickel_file(pickel_path=save_dictionaries_path,
                           pickel_filename=(pickel_filename + "arc_dict.pickle"),
                           pickel_dictionary=final_pickel)
        print("File saved :) - With exceptions :( ")

File saved :)


In [67]:
pickel_dict

{'f_s_av': 285.86545,
 'f_m_av': 274.2673,
 'f_e_av': 246.52644,
 'f_s_stdv': 196.01395,
 'f_m_stdv': 192.15654,
 'f_e_stdv': 179.38641,
 'zodi': 1.5130043116912668e-18}